In [ ]:
## This code was last modified on 27/11/2025

## The code calculates the number of Pauli strings required to encode the 0+1 supersymmetric Hamiltonian
## for three supersymmetric potentials (HO, DW, and AHO). It then fits the data using the following function: y = a + b*x + c*x*log2(x)

In [ ]:
# Essential information about the structure of the code for each block
"""
Code blocks structure
1) Load all libraries module
2) User choices
3) Custom functions needed
4) Calculate the pauli strings
5) Fitting with y = a + b*x + c*x*log2(x)
6) log-log plot

"""

In [ ]:
## Section importing all needed libraries modules

import sys
#Append the code's directory to the interpreter to search for files
sys.path.append('..')

#Importing the Hamiltonian from the file (Hamiltonian_SQM_0p1.py) inside the folder (hamiltonian_SQM)
from hamiltonian_SQM.Hamiltonian_SQM_0p1 import *

# Needed to convert the Hamiltonian into pauli gates
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp

#Packages for interacting with the operating system
import os

#Packages for numerical calculation
import statistics
import numpy as np

import matplotlib.pyplot as plt


from scipy.sparse.linalg import eigs

#For eigenvalues and eigenvectors
from numpy import linalg 


#pakages needed 
from scipy.sparse import diags_array
from scipy.sparse.linalg import matrix_power

# Package needed for the fit
from scipy.optimize import curve_fit

In [ ]:
## Define user choice

# Parameters for the construction of the Hamiltonian
superpotential = "AHO"                   # Chose a superpontentials "HO" Harmonic Oscillator; "DW" Double Well; "AHO" Anharmonic Oscillator 
m = 1                                   # Chose the bosonic and fermionic mass
g = 1                                   # Chose the g interaction strenght present in AHO and DW
mu = 1                                  # Chose the mu interaction strenght present in DW

H_full_or_block = "Full"                # Chose the "Full" for the full Hamiltonian or "Block" for the lower block

In [ ]:
#Defining all needed functions

# Function to create a compact string version for number in scientific notation 
def SnString(number):
    # Convert a number in scientific notation
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'e' + a.split('E')[1]
#============================================================================================================

# Function to convert a numbers in scientific notation using latex
def latex_number_SN(number):
    a = '%.4E' % number
    return "$"+str(a.split('E')[0].rstrip('0').rstrip('.'))+ "\u005c" + "mathrm{e}"+ "{"+str(a.split('E')[1])+"}"+ "$"

#============================================================================================================


# Function used for fitting the Npaulis y = a + b*x + c*x*log2(x)
def model(x, a, b, c):
    return a + b * x + c * x * np.log2(x)

#=============================================================================================================


# Function used for the log-log plot Npaulis y = 2^( x*a + b - c*x*log2(x) )
def plotting_function(x, a, b, c):
    return 2**(x*a + b - c * np.log2(x))

#=============================================================================================================

In [ ]:
#Calculate the number of pauli string for the q and p operators in the Fock basis:

N_bosons_list = [2**(n) for n in range(1, 13 + 1)]

print(r"Lambda     matrix_size       N_qubits      N_paulis")

latex_string_list=[]

pauli_list=[]

N_qubits_list=[]

for i in N_bosons_list:

    # Calculate the Hamiltonian
    # Calculate the Hamiltonian 
    hamiltonian = Hamiltonian_SQM_0p1(superpotential, n_bosonic_modes = i, m = m, g = g, mu = mu)

    #Dictionary to handle the selection of full or block Hamiltonian 
    N_bosonic_modes=i
    H_full_or_block_dict = {
        "Full": hamiltonian.toarray(),
        "Block": hamiltonian.toarray()[i:2*i,i:2*i]
        }

    hamiltonian_array = H_full_or_block_dict.get(H_full_or_block, "Choice not present! type " + str([i for i in H_full_or_block_dict.keys()]))


    Hamiltonian_op = SparsePauliOp.from_operator(hamiltonian_array, atol = 1e-18)
        
    N_qubits = Hamiltonian_op.num_qubits

    N_paulis = Hamiltonian_op.size

    matrix_size = len(hamiltonian_array)

    pauli_list.append(N_paulis)

    N_qubits_list.append(N_qubits)
    
    print(f"{i}     {matrix_size}       {N_qubits}      {N_paulis}")
    
    latex_string_list.append(f"{i}" +" & "+f"${matrix_size}" + "\u005c"+"times " + f"{matrix_size}$" +" & " + f"{N_qubits}" + " & "+ f"{N_paulis}" +"\u005c"+"\u005c")


# Print all lines inside a file
with open(os.path.join(f"LateX_C_pauli_"+superpotential+"_sho_tol_em18_m_{m}__g_{g}_.txt"), 'a') as file:                            
            for row in latex_string_list:
                    print(row, file = file)
file.close()


In [ ]:
print("Fit H operator pauli string with the following curve: y = a + b*x + c*x*log2(x)")

#Select the pointa for the fitting, 0 --> all points, 1--> cut the first point and so on.
initial_index= 3


# Enter the data here
x_data = np.array([1 / a for a in  N_qubits_list])[initial_index:]
y_data = np.array([np.log2(a) / b for a, b in zip(pauli_list, N_qubits_list)])[initial_index:]


# Fit the data to the model using curve_fit
popt, pcov = curve_fit(model, x_data, y_data, p0=[1, 1, 1])

# popt contains the optimized parameters [a, b, c]
a_fit, b_fit, c_fit = popt

# The standard errors of the fitted parameters are the square root of the diagonal elements of the covariance matrix
parameter_errors = np.sqrt(np.diag(pcov))

# Extract standard errors for each parameter
a_error, b_error, c_error = parameter_errors

# Predicted y values from the model
y_pred = model(x_data, *popt)

# Residuals (difference between observed and predicted values)
residuals = y_data - y_pred

# R-squared
ss_res = np.sum(residuals**2)  # Residual sum of squares
ss_tot = np.sum((y_data - np.mean(y_data))**2)  # Total sum of squares
r_squared = 1 - (ss_res / ss_tot)

# Mean Squared Error (MSE)
mse = np.mean(residuals**2)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate the residual standard devition
std_residuals  = np.std(residuals)


# Output the metrics

print(f"Initial x point: {initial_index}")
# Print the results
print(f"R-squared: {r_squared:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")

print("\n")

# Output the coefficients and their errors
print(f"Optimized parameters: a = {a_fit:.4f} ± {a_error:.4f}, b = {b_fit:.4f} ± {b_error:.4f}, c = {c_fit:.4f} ± {c_error:.4f}")

# Output the line's equation
print(f"Curve equation: y = {a_fit:.4f} + {b_fit:.4f}*x + {c_fit:.4f}*x*log2(x)")

#=============================================================================================


# Predicted y values from the model
y_pred = model(x_data, *popt)

# Set plot size
plt.figure(figsize=(10,10))

# Enable LaTeX-style formatting globally
plt.rcParams['text.usetex'] = True

#Plot title and axis labels
fontsize = 22

# Plot the data and the fitted curve
plt.scatter(x_data, y_data, label="Data", color='blue', zorder=3)


#Plot fitted curve
# Generate a range of x-values for a smooth curve (continuous way)
x_fine = np.linspace(1e-6, max(x_data), 500) #min(x_data)
y_fine = model(x_fine, *popt)
plt.plot(x_fine, y_fine, label="Fitted curve", color='red', linewidth=2, zorder=2)



#plt.title("Anharmonic Oscillator  "+ r"$(\frac{1}{2}m\hat{q}^2 + \frac{1}{4} g \hat{q}^4)$"  +"   Fitting function  "+ "y = a + b*x + c*x*log2(x)", fontsize = fontsize)

# Ticks for x-axis
plt.xticks(fontsize = fontsize) #x_data,

# Ticks for y-axis
plt.yticks(fontsize = fontsize) 

# Add labels and title
plt.xlabel(r'$1/Q$', fontsize = 30)

plt.ylabel(r"$\log_2(N_{\rm Pauli}) /Q$", fontsize = 30)

#plt.title('Curve Fitting: y = a + b*x + c*x*log2(x)')


#==========================================
# Create a confidence band (we use 95% confidence level here)
confidence_interval = 1.96 * std_residuals  # 95% confidence interval (approx. 1.96 * std dev)

# Plot the upper and lower bounds of the confidence band
plt.fill_between(x_fine, y_fine - confidence_interval, y_fine + confidence_interval, color='gray', alpha=0.2, label=r"Confidence band 95$\%$", zorder=2)

#==========================================


#==========================================
# x ticks
plt.xticks([0.00, 0.05, 0.1, 0.15, 0.2])

#plt.xlim(0, 0.25)
#==========================================

#plt.axvline(x=0, color='black', linestyle='-', linewidth=2)

plt.grid(True,zorder=1)

plt.legend(fontsize = 18, loc='center right')

plt.savefig('Paulis_C_AHO_log2_latex_Q5_Large_600.png', bbox_inches='tight', format='png', dpi=600)

plt.show()


In [ ]:
## In this section we use a log-log plot

print("Fit H operator pauli string with the following curve: y = a*x + b - c*log2(1/x)")

#Select the pointa for the fitting, 0 --> all points, 1--> cut the first point and so on.
initial_index= 0


# Enter the data here
x_data_new = np.array([a for a in  N_qubits_list])[initial_index:]
y_data_new = np.array([a for a in pauli_list])[initial_index:]


# Set plot size
plt.figure(figsize=(10,10))

# Enable LaTeX-style formatting globally
plt.rcParams['text.usetex'] = True

#Plot title and axis labels
fontsize = 35


# Set logarithmic scale for x and y axes with base 2
#plt.xscale('log', base=2)
plt.yscale('log', base=2)

# Plot the data and the fitted curve
#Plot the data 
plt.scatter(x_data_new, y_data_new, label="Data", color='blue', s=100, zorder=3)



#plt.title("Anharmonic Oscillator  "+ r"$(\frac{1}{2}m\hat{q}^2 + \frac{1}{4} g \hat{q}^4)$", fontsize = fontsize)

# Ticks for x-axis
plt.xticks(fontsize = fontsize) #x_data,

# Ticks for y-axis
plt.yticks(fontsize = fontsize) 

# Add labels and title
plt.xlabel(r'$N_{\rm qubits}$', fontsize = 50, labelpad=10)

plt.ylabel(r"$N_{\rm Paulis}$", fontsize = 50, labelpad=10)



#Plot 2^x
x_2tox = np.linspace(1, max(x_data_new), 1000) 

#Plot fitted curve calculated in the previous step
y_fit_function = plotting_function(x_2tox, *popt)

plt.plot(x_2tox, y_fit_function, label="Fitted curve", color='red', linewidth=5, zorder=2)


"""
#==========================================
# Plot the upper and lower bounds of the confidence band
# Create a confidence band (we use 95% confidence level here)
confidence_interval = 1.96 * std_residuals  # 95% confidence interval (approx. 1.96 * std dev)

plt.fill_between(x_2tox, y_fit_function - confidence_interval, y_fit_function + confidence_interval, color='gray', alpha=0.2, label=r"Confidence band 95$\%$", zorder=1)
#==========================================
"""

plt.grid(True,zorder=1)

plt.legend(fontsize = 40, loc='upper left')

plt.savefig("Paulis_C_"+superpotential+"_loglog_latex_Large_600_personal_.pdf", bbox_inches='tight', format='pdf', dpi=600)

plt.show()
